In [3]:
import os
import sys
import asyncio
import json
from typing import Any, Dict, List
from pathlib import Path
# Add the src directory to Python path for imports
sys.path.insert(0, str(Path.cwd().parent / 'src'))
from subagent.eda_agent import EDAAgent
from agent import Agent
from agentdk.core.logging_config import ensure_nest_asyncio


def get_llm():    
    # Try OpenAI
    if os.getenv('OPENAI_API_KEY'):
        try:
            from langchain_openai import ChatOpenAI
            model = "gpt-4o-mini"
            llm = ChatOpenAI(model=model, temperature=0)
            print(f"✅ Using OpenAI {model}")
            return llm
        except ImportError:
            print("❌ langchain_openai not available")
        except Exception as e:
            print(f"❌ OpenAI setup failed: {e}")
    
    # Try Anthropic
    elif os.getenv('ANTHROPIC_API_KEY'):
        try:
            from langchain_anthropic import ChatAnthropic
            llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
            print("✅ Using Anthropic Claude-3-Haiku")
            return llm
        except ImportError:
            print("❌ langchain_anthropic not available")
        except Exception as e:
            print(f"❌ Anthropic setup failed: {e}")

    else:
        raise ValueError("No LLM API key found")
    # Fallback to mock
    return llm

# Get LLM instance
llm = get_llm()



✅ Successfully imported AgentDK components
Environment setup complete!
✅ Using OpenAI gpt-4o-mini


In [4]:
agent = Agent(llm)
print(agent("which table i have access to?"))
print(agent("what is the total transaction amount from customer 'John Smith'"))

2025-06-23 19:03:02,994 - agentdk - INFO - Loaded MCP configuration from: subagent/mcp_config.json
2025-06-23 19:03:02,995 - agentdk - INFO - MCP client configured with 1 servers
2025-06-23 19:03:03,361 - agentdk - INFO - Loaded 1 tools from MCP servers
2025-06-23 19:03:03,366 - agentdk - INFO - Created LangGraph agent with 1 tools
2025-06-23 19:03:03,366 - agentdk - INFO - Agent EDAAgent initialized successfully
2025-06-23 19:03:04,004 - agentdk - INFO - {"tool": "execute_sql", "args": {"query": "SHOW TABLES;"}}
2025-06-23 19:03:04,402 - agentdk - INFO - {"tool": "execute_sql", "status": "completed"}
```sql
SHOW TABLES;
```

**Available Tables:**
- accounts
- customer_account_summary  
- customers
- monthly_transaction_summary
- transactions
2025-06-23 19:03:08,733 - agentdk - INFO - {"tool": "execute_sql", "args": {"query": "SELECT SUM(amount) AS total_amount \nFROM transactions t\nJOIN accounts a ON t.account_id = a.account_id\nJOIN customers c ON a.customer_id = c.customer_id\nWHER